# **Actividad 2**: Convolutional Neural Networks (CNNs)

En la primera parte de esta actividad veremos los conceptos necesarios para implementar una Convolutional Neural Network usando Keras. Luego, tendreis que entrenar una CNN para que resuelva la tarea de clasificación de imágenes del dataset Fashion MNIST.

**EVALUACIÓN**: En esta segunda práctica, tendreis completar el esqueleto del notebook y redactar un informe de 2/3 páginas explicando:
* La arquitectura del modelo y la justificación de las decisiones de diseño.
* El impacto del aumento de datos en el rendimiento del modelo.
* El impacto del uso de transfer learning y de un modelo preentrenado.
* Los desafíos encontrados durante la implementación y cómo se resolvieron.
* La precisión final obtenida y observaciones sobre el rendimiento del modelo comparado con las previas implementaciones con la FNN.

La nota de la actividad esta divida en un 50% para la evaluación del notebook y un 50% para la evaluación del informe.

In [1]:
from __future__ import absolute_import, division, print_function
from keras.layers import Activation, Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

## Implementando una CNN para Image Classification

## Background

Con la siguiente función creamos una capa Conv2D con 32 filtros, cada uno de 3 × 3, usando un stride de 1 (tanto horizontal como vertical) y padding "same", y aplicando la función de activación ReLU a sus salidas.

`conv = keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu")`


Por otro lado, la siguiente función crea una capa de max pooling usando un kernel de 2 × 2. Los strides por defecto son del tamaño del kernel, por lo que esta capa usará un stride de 2 (tanto horizontal como verticalmente). Por defecto, utiliza padding "valid" (es decir, sin padding):

```
max_pool = keras.layers.MaxPool2D(pool_size=2)
```
Para crear una capa de average pooling, simplemente usaremos AvgPool2D en lugar de MaxPool2D.







In [2]:
avg_pool = keras.layers.AvgPool2D(pool_size=2)

Las arquitecturas típicas de CNN apilan unas cuantas capas convolucionales (cada una generalmente seguida por una capa ReLU), luego una capa de pooling, luego otras pocas capas convolucionales (+ReLU), luego otra capa de pooling, y así sucesivamente. La imagen se hace cada vez más pequeña a medida que avanza a través de la red, pero también generalmente se hace más profunda (es decir, con más filtros), gracias a las capas convolucionales. La capa final produce la predicción (en el caso de clasifiación le aplicamos una capa softmax que da como resultado las probabilidades de las clases).




In [3]:
"""
model = keras.models.Sequential([
       keras.layers.Conv2D(..., ..., activation="relu", padding="same", input_shape=...),
       keras.layers.MaxPooling2D(...),
       keras.layers.Conv2D(..., ..., activation="relu", padding="same"),
       keras.layers.Conv2D(..., ..., activation="relu", padding="same"),
       keras.layers.MaxPooling2D(...),
       keras.layers.Flatten(),
       keras.layers.Dense(..., activation="relu"),
       keras.layers.Dropout(...),
       keras.layers.Dense(..., activation="softmax")
])
"""

'\nmodel = keras.models.Sequential([\n       keras.layers.Conv2D(..., ..., activation="relu", padding="same", input_shape=...),\n       keras.layers.MaxPooling2D(...),\n       keras.layers.Conv2D(..., ..., activation="relu", padding="same"),\n       keras.layers.Conv2D(..., ..., activation="relu", padding="same"),\n       keras.layers.MaxPooling2D(...),\n       keras.layers.Flatten(),\n       keras.layers.Dense(..., activation="relu"),\n       keras.layers.Dropout(...),\n       keras.layers.Dense(..., activation="softmax")\n])\n'

**Conceptos**

* Siempre usaremos un tamaño de kernel impar para que esté centrado en un píxel y el kernel se aplique de forma simétrica. Es preferible usar un tamaño de kernel pequeño (3x3) y apilar varias capas convolucionales, en vez de usar un tamaño de kernel mas grande (5x5), ya que podemos obtener una mejora similar con menos parámetros. En el caso de la primera capa conectada con el input, si que podemos usar un kernel mas grande con stride para reducir la dimensionalidad de la imágen.
*  Incrementamos el número de kernels en cada capa convolucional a medida que esta se hace más profunda, ya que el número de características de bajo nivel en una imagen (vértices, lineas horizontales, círculos) es menor que el número de características de alto nivel (características complejas de la imagen que son combinacion de características de bajo nivel).

*  Una pooling layer con una pool size de 2 divide cada dimension espacial entre 2.







## Actividad

Las siguientes tareas ayudan a construir el esqueleto de una implementación básica de una CNN, pero las podeis adaptar a vuesta propia implementación. Para tener una buena performance tendreis que hacer encontrar los valores óptimos de los hiperparámetros de la red, y experimentar con distintas técnicas (regularización, dropout...).

**Tarea 1:** Cargar y preprocesar el dataset.

In [4]:
# Cargar el dataset MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar las imágenes a valores entre 0 y 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Añadir una dimensión extra para el canal (ya que las imágenes son en escala de grises)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Verificar las formas de los datos
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
x_test shape: (10000, 28, 28, 1)
y_test shape: (10000,)


**Tarea 2:** Definir la arquitectura del modelo.


In [5]:
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

**Tarea 3:** Compilar el modelo.


In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**Tarea 4:** Aumentar los datos.


In [7]:
"""
[TIP] Utiliza ImageDataGenerator de Keras para aplicar técnicas de aumento de datos.
"""
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(x_train)


**Tarea 5:** Entrenar el modelo.





In [8]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 95s 49ms/step - loss: 0.2398 - accuracy: 0.9243 - val_loss: 0.0465 - val_accuracy: 0.9853
Epoch 2/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.0832 - accuracy: 0.9742 - val_loss: 0.0278 - val_accuracy: 0.9906
Epoch 3/10
1875/1875 [==============================] - 71s 38ms/step - loss: 0.0621 - accuracy: 0.9808 - val_loss: 0.0375 - val_accuracy: 0.9894
Epoch 4/10
1875/1875 [==============================] - 69s 37ms/step - loss: 0.0526 - accuracy: 0.9839 - val_loss: 0.0300 - val_accuracy: 0.9903
Epoch 5/10
1875/1875 [==============================] - 69s 37ms/step - loss: 0.0449 - accuracy: 0.9868 - val_loss: 0.0316 - val_accuracy: 0.9899
Epoch 6/10
1875/1875 [==============================] - 72s 38ms/step - loss: 0.0417 - accuracy: 0.9871 - val_loss: 0.0172 - val_accuracy: 0.9945
Epoch 7/10
1875/1875 [==============================] - 69s 37ms/step - loss: 0.0396 - accuracy: 0.9883 - val_loss: 0.0203 -

**Tarea 6:** Evaluar el modelo.


In [9]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 3s 8ms/step - loss: 0.0219 - accuracy: 0.9923
Test accuracy: 0.9922999739646912


**Tarea 7**: **Transfer learning**. Importar un modelo preentrenado (e.gVGG16 o ResNet50) y adaptar su arquitectura para Fashion MNIST. Evaluar la performance del modelo, y comparalo con la CNN anterior.